# Reinforcement Learning
Prof. Milica Gašić

### TD control

In this notebook we will implement the TD control algorithms SARSA and Q-learning.  
The main difference between them is the TD target, e.g. for SARSA:

$\begin{equation}
    Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha (\,\underbrace{r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})}_{\text{TD target}} - Q(s_t, a_t))
\end{equation}$

The target of Q-learning can be found in the slides.  
Another difference is whether the next action is already selected before the update (only SARSA).

#### Implementation

Make sure that the files `rl_agent.py` and `rl_env.py` are in the same folder as the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import rl_agent
import rl_env

#### Agents

The class `TDAgent` is a base class and uses the method `compute_target()` to compute the TD target.  
The two TD agents are subclasses of `TDAgent` and only need to implement this method.  
Follow the instructions in the code cells below.

In [ ]:
class TDAgent(rl_agent.TabularAgent):

    def __init__(self, env, gamma, epsilon, alpha, rng=None):
        super().__init__(env)
        self.gamma = gamma      # discount factor
        self.epsilon = epsilon  # epsilon-greedy probability
        self.alpha = alpha      # learning rate
        self.rng = rng if rng is not None else np.random.default_rng()
        self.reset()

    def reset(self):
        # Create array for action values
        self.q = np.zeros((self.num_states, self.num_actions), dtype=float)
        # Create array for policy distribution (initialized uniformly)
        self.pi = np.full((self.num_states, self.num_actions), 1 / self.num_actions)

    def policy(self, state):
        # Sample an action according to pi
        action_probs = self.pi[state]
        action = self.rng.choice(self.num_actions, p=action_probs)
        return action

    def value(self, state):
        # Compute the state value from q and pi
        return np.sum(self.pi[state] * self.q[state])

    def action_value(self, state, action):
        # Lookup the action value in q
        return self.q[state, action]

    def policy_evaluation(self, state, action, reward, next_state, terminated, truncated):
        #######################################################################
        # TODO Perform one step of policy evaluation using compute_target().  #
        # Update the action value for the given transition. Remember to check #
        # if the episode is terminated. Note that SARSA already selects the   #
        # next action, so in this case the method should return the next      #
        # action, in order to perform it in the environment.                  #
        #######################################################################
        
        #######################################################################
        # End of your code.                                                   #
        #######################################################################

    def compute_target(self, reward, next_state):
        # Compute the TD target and the next action, if necessary.
        # Needs to be implemented in a subclass.
        raise NotImplementedError()

    def policy_improvement(self):
        # Update the policy using epsilon-greedy policy improvement
        q = self.q
        epsilon = self.epsilon

        # Find the maximizing actions
        max_q = np.max(q, axis=1, keepdims=True)
        max_mask = q == max_q
        num_max = np.sum(max_mask, axis=1, keepdims=True)

        # Compute the probabilities
        num_actions = q.shape[1]
        pi = ((1 - epsilon) / num_max) * max_mask
        pi += epsilon / num_actions
        self.pi = pi

In [ ]:
class SarsaAgent(TDAgent):

    def compute_target(self, reward, next_state):
        #######################################################################
        # TODO Calculate the SARSA target. Remember that this algorithm       #
        # already selects the next action, so it should be returned as well.  #
        #######################################################################
        
        #######################################################################
        # End of your code.                                                   #
        #######################################################################

In [ ]:
class QLearningAgent(TDAgent):

    def compute_target(self, reward, next_state):
        #######################################################################
        # TODO Calculate the Q-learning target.                               #
        #######################################################################
        
        #######################################################################
        # End of your code.                                                   #
        #######################################################################

#### Evaluation

Now we evaluate the TD agents on the cliff walking environment.  
The goal is to recreate Figure 6.3 from the textbook http://incompleteideas.net/book/the-book-2nd.html  
which compares different learning rates $\alpha$. This week, we will plot the lines for Sarsa and Q-learning only.

First implement `train_td_agent()`, which learns a policy.

In [ ]:
def train_td_agent(env, agent, num_episodes):
    #######################################################################
    # TODO Train a given TD agent by collecting `num_episodes` episodes   #
    # in the given environment and performing policy evaluation and       #
    # policy improvement.                                                 #
    #######################################################################
    
    #######################################################################
    # End of your code.                                                   #
    #######################################################################

Now implement `evaluate_td_agent()`, which evaluates a learned policy.

In [ ]:
def evaluate_td_agent(env, agent, num_episodes):
    #######################################################################
    # TODO Evaluate a trained TD agent by collecting `num_episodes`       #
    # episodes and calculating the average sum of rewards per episode.    #
    #######################################################################
    
    #######################################################################
    # End of your code.                                                   #
    #######################################################################

Now we evaluate the TD agents in the cliff walking environment.  
We want to compare their results when using different learning rates $\alpha \in \{0.1, 0.4, 0.7, 1.0\}$,  
so you need to train and evaluate each agent four times.  
Always use $\gamma = 1.0$ and $\epsilon = 0.1$, and train and evaluate for $3000$ episodes.

In [ ]:
# Create a cliff walking environment
env = rl_env.cliff_walking()
gamma = 1.0    # discount factor
epsilon = 0.1  # epsilon-greedy probability
alphas = [0.1, 0.4, 0.7, 1.0]  # learning rates
num_train_episodes = 3000  # number of episodes for training
num_eval_episodes = 3000   # number of episodes for evaluation

#######################################################################
# TODO Train and evaluate the two agents as described above. Plot   #
# the resulting sums of rewards per episode, to recreate Figure 6.3   #
# from the textbook.                                                  #
#######################################################################

#######################################################################
# End of your code.                                                   #
#######################################################################

What do you observe?